In [1]:
!pip install transformers peft torch datasets accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!huggingface-cli login
# from huggingface_hub import notebook_login
# notebook_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `test` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `test`


In [3]:
# 3 hours spent trying to figure out how to fine-tune with insufficient vram.
# the solution? just pay for a better colab GPU, apparently

In [4]:
import json
from datasets import Dataset

# Load JSON data
def read_in_data(json_file):
    with open(json_file, "r") as f:
        return json.load(f)

# Read and format dataset
JSON_FILE = "prompts_sample_n1.json"
jdata = read_in_data(JSON_FILE)

dataset = Dataset.from_dict({"text": [d['prompt'] + d['ground_truth'] for d in jdata[5:35]]})

# Split dataset for training & evaluation
dataset = dataset.train_test_split(test_size=0.2)


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# Load model & tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DialogueDiscriminator(nn.Module):
    def __init__(self, hidden_dim=256, input_dim=4096):  # Mistral hidden dim is 4096
        super(DialogueDiscriminator, self).__init__()
        self.encoder = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim,
                               batch_first=True, bidirectional=True)
        self.fc = nn.Sequential(
            nn.Linear(2 * hidden_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        _, (h_n, _) = self.encoder(x)
        h = torch.cat((h_n[-2], h_n[-1]), dim=1)
        return self.fc(h)

# 2. Instantiate discriminator
input_dim = model.model.embed_tokens.embedding_dim  # Get from current transformer
hidden_dim = 256
discriminator = DialogueDiscriminator(hidden_dim=hidden_dim, input_dim=input_dim).to("cuda")
discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=1e-5)


In [7]:
def bce_loss(preds, targets):
  loss_fn = nn.BCELoss()
  return loss_fn(preds, targets)

In [8]:
print(model)
# 32 mistralattn layers, mlp

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4096,), eps=1e-0

In [9]:
for param in model.parameters():
    param.requires_grad = False

for layer in model.model.layers[-2:]:
    for param in layer.parameters():
        param.requires_grad = True


trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters: {trainable_params}")

Trainable parameters: 436224000


In [10]:
tokenizer.pad_token = tokenizer.unk_token

def tokenize_function(examples):
    tokenized = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)
    tokenized["labels"] = tokenized["input_ids"].copy()  # Labels should be the same as input_ids
    return tokenized

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/24 [00:00<?, ? examples/s]

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

In [11]:
def generate_response(prompt):
  model.eval()
  with torch.no_grad():
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_new_tokens=100)
    return tokenizer.decode(output[0], skip_special_tokens=True)
generate_response("Hello!")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"Hello! I'm a new member of the community and I'm excited to be here. I'm a writer and I love to read and write about science fiction and fantasy. I'm also a huge fan of video games and I love to play them. I'm always looking for new games to play and new stories to read. If you have any recommendations, please let me know!"

In [12]:
for i, (b, a) in enumerate(zip(before_weights, after_weights)):
    if not torch.allclose(b, a):
        print(f"Parameter {i} changed!")
    else:
      print(i)

NameError: name 'before_weights' is not defined

In [13]:
def embed_texts(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=256).to("cuda")
    with torch.no_grad():
        return model.model.embed_tokens(inputs['input_ids'])

In [16]:
from accelerate import Accelerator
from torch.utils.data import DataLoader
from torch.optim import AdamW
import os

lr = 1e-4
epochs = 3
batch_size = 16


accelerator = Accelerator(mixed_precision="no")
train_loader = DataLoader(tokenized_datasets['train'], batch_size=batch_size, shuffle=True)
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
# thank you, adamw, for randomly deciding to replace entire matrices of weights
# with nan. thank you for not raising any sort of error when you did that, too.

model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader)
loss_history = []
model.train()
for epoch in range(epochs):
    for batch in train_loader:
        outputs = model(**batch)
        loss = outputs.loss

        if not torch.isfinite(loss):
          raise Exception("wee woo wee woo infinite loss alert")
        loss_history.append(loss.item())
        layer = model.model.layers[-1]

        accelerator.backward(loss)

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        optimizer.zero_grad()
        # raise Exception("wawa")
print("loss", loss_history)

loss [2.560574531555176, 2.4280149936676025, 2.5093703269958496, 2.5296709537506104, 2.576374053955078, 2.3949782848358154]


In [17]:


for epoch in range(10):
    for batch in train_loader:
        #print(batch)
        real_texts = tokenizer.batch_decode(batch['input_ids'], skip_special_tokens=True)
        fake_texts = [generate_response(prompt) for prompt in real_texts]

        # Embed
        real_embeds = embed_texts(real_texts).float()
        fake_embeds = embed_texts(fake_texts).float()

        # Discriminator training
        d_real = discriminator(real_embeds)
        d_fake = discriminator(fake_embeds)

        d_loss_real = bce_loss(d_real, torch.ones_like(d_real))
        d_loss_fake = bce_loss(d_fake, torch.zeros_like(d_fake))
        d_loss = d_loss_real + d_loss_fake

        discriminator_optimizer.zero_grad()
        d_loss.backward()
        discriminator_optimizer.step()

        # Generator training (language model)
        fake_embeds = embed_texts([generate_response(prompt) for prompt in real_texts]).float()
        g_fake = discriminator(fake_embeds)
        g_loss = bce_loss(g_fake, torch.ones_like(g_fake))

        model.zero_grad()
        g_loss.backward()
        optimizer.step()

        print(f"Epoch {epoch} | D_loss: {d_loss.item():.4f} | G_loss: {g_loss.item():.4f}")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

Epoch 0 | D_loss: 1.3875 | G_loss: 0.7286


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

Epoch 0 | D_loss: 1.3875 | G_loss: 0.7284


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

...

Epoch 8 | D_loss: 1.3872 | G_loss: 0.7242


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

Epoch 9 | D_loss: 1.3872 | G_loss: 0.7240


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

Epoch 9 | D_loss: 1.3872 | G_loss: 0.7238


In [18]:
for j in jdata[:5]:
  prompt = j['prompt']
  print("===========================================")
  print(prompt)
  print(generate_response(prompt))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Character Info: Tony Last: The novel's protagonist, a gentleman devoted to his estate, Hetton Abbey. His naive attachment to tradition and his wife, Brenda, leads to his ultimate downfall.
John Andrew: Tony and Brenda Last's young son, whose tragic death marks the turning point of the novel and accelerates Tony's misfortune.


Dialogue History: John Andrew: Is mummy coming down to-day, daddy?
Tony Last: Yes.
John Andrew: Who else?
Tony Last: Someone called Jenny Abdul Akbar.
John Andrew: What a silly name. Is she foreign?
Tony Last: I don't know.
John Andrew: Sounds foreign, doesn't she, daddy? D'you think she won't be able to talk any English? Is she black?
Tony Last: Mummy says not.
John Andrew: Oh... who else?
Tony Last: Lady Cockpurse.

Instruction: Continue the dialogue for 1 lines. 
 [/INST]


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Character Info: Tony Last: The novel's protagonist, a gentleman devoted to his estate, Hetton Abbey. His naive attachment to tradition and his wife, Brenda, leads to his ultimate downfall.
John Andrew: Tony and Brenda Last's young son, whose tragic death marks the turning point of the novel and accelerates Tony's misfortune.


Dialogue History: John Andrew: Is mummy coming down to-day, daddy?
Tony Last: Yes.
John Andrew: Who else?
Tony Last: Someone called Jenny Abdul Akbar.
John Andrew: What a silly name. Is she foreign?
Tony Last: I don't know.
John Andrew: Sounds foreign, doesn't she, daddy? D'you think she won't be able to talk any English? Is she black?
Tony Last: Mummy says not.
John Andrew: Oh... who else?
Tony Last: Lady Cockpurse.

Instruction: Continue the dialogue for 1 lines. 
 [/INST] John Andrew: What's she like, daddy?
[INST] Character Info: Marjorie: A peripheral character who briefly appears in the social setting, emphasizing the transience of social relationshi

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Character Info: Marjorie: A peripheral character who briefly appears in the social setting, emphasizing the transience of social relationships.
Jock Grant-Menzies: A close friend of Tony Last who offers support but ultimately fails to prevent Tony's personal collapse, representing the limits of friendship in crises.


Dialogue History: Marjorie: D'you think Tony knows about Mr Beaver?

Instruction: Continue the dialogue for 1 lines. 
 [/INST] Jock Grant-Menzies: I don't know, but if he does, he'll probably just shrug it off. Tony's been through a lot lately.
[INST] Character Info: Mr Graceful: A minor character known for his polished manners and charming personality, often blending into the upper-class crowd.
Brenda: Tony Last's wife, who becomes dissatisfied with her marriage and has an affair with John Beaver, ultimately prioritizing her own social standing over her marriage.


Dialogue History: Brenda: Mr Graceful, I've got to have some more money.

Instruction: Continue the 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Character Info: Mr Graceful: A minor character known for his polished manners and charming personality, often blending into the upper-class crowd.
Brenda: Tony Last's wife, who becomes dissatisfied with her marriage and has an affair with John Beaver, ultimately prioritizing her own social standing over her marriage.


Dialogue History: Brenda: Mr Graceful, I've got to have some more money.

Instruction: Continue the dialogue for 1 lines. 
 [/INST] Mr Graceful: Of course, my dear. How much do you need?
[INST] Character Info: Nanny: A nurturing and steady figure in Tony Last's household, providing a stable presence in the tumultuous events of the family.
Jenny Abdul Akbar: A minor but colorful character, known for her exotic lifestyle and diverse social connections. She frequently appears at social events and embodies a cosmopolitan worldliness.
Tony Last: The novel's protagonist, a gentleman devoted to his estate, Hetton Abbey. His naive attachment to tradition and his wife, Bre

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Character Info: Nanny: A nurturing and steady figure in Tony Last's household, providing a stable presence in the tumultuous events of the family.
Jenny Abdul Akbar: A minor but colorful character, known for her exotic lifestyle and diverse social connections. She frequently appears at social events and embodies a cosmopolitan worldliness.
Tony Last: The novel's protagonist, a gentleman devoted to his estate, Hetton Abbey. His naive attachment to tradition and his wife, Brenda, leads to his ultimate downfall.
Brenda: Tony Last's wife, who becomes dissatisfied with her marriage and has an affair with John Beaver, ultimately prioritizing her own social standing over her marriage.
John Andrew: Tony and Brenda Last's young son, whose tragic death marks the turning point of the novel and accelerates Tony's misfortune.
Polly Cockpurse: A lively socialite who thrives in London's high society. Known for her gossiping nature and tendency to interfere in the affairs of others.


Dialogue 

In [19]:
# Save
#model.save_pretrained("./gan_fine_tuned_model")
#tokenizer.save_pretrained("./gan_fine_tuned_model")

# Load
# from peft import PeftModel

# base_model = AutoModelForCausalLM.from_pretrained(model_name)
# fine_tuned_model = PeftModel.from_pretrained(base_model, "./fine_tuned_model")


In [20]:
filename = "prompts_sample_n1.json"
with open(filename, "r", encoding="utf-8") as f:
  data = json.load(f)
print(f"Loaded {len(data)} prompts.")

Loaded 200 prompts.


In [21]:
pairs = []
clean_output = []
tokenizer.pad_token_id = tokenizer.eos_token_id
for index, item in enumerate(data[:200]):
    if index % 100 == 0:
        print(f"Processed {index} prompts.")

    prompt = item.get("prompt", "").strip()
    instruction = item.get("Instruction", "").strip()
    full_prompt = prompt + ("\n" + instruction if instruction else "")
    completion = item.get("ground_truth", "").strip()


    prediction = item.get("predicted_output", "").strip()


    tokenized = tokenizer(full_prompt, return_tensors="pt")
    input_ids = tokenized.input_ids.to(model.device)
    attn_mask = tokenized.attention_mask.to(model.device)
    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids, attention_mask=attn_mask, max_new_tokens=50)  # Adjust max_new_tokens as needed
    predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)[len(full_prompt):]
    clean_output.append(predicted_text.split('\n')[0])

    pairs.append((full_prompt, completion, predicted_text))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Processed 0 prompts.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

Processed 100 prompts.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

In [22]:
with open("pairs_tuned_full_gan.json", "w") as f:
    json.dump(pairs, f)

In [23]:
for i,o in enumerate(clean_output):
  pairs[i] = (pairs[i][0], pairs[i][1], o)
with open("pairs_tuned_clean_full_gan.json", "w") as f:
    json.dump(pairs, f)

In [24]:
filename = "pairs_tuned_full_gan.json"
with open(filename, "r", encoding="utf-8") as f:
    pairs = json.load(f)

In [25]:
!pip install rouge_score transformers peft torch datasets accelerate bitsandbytes sentence_transformers evaluate rouge nltk

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.5 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=dec28fdb6e0b4b8d67bbd452aad17f594e99e7bc7809458e05595bedbd8a36c2
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [26]:
import evaluate
import numpy as np
from tqdm import tqdm
# Load the metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")
bleu_scores = []
rouge_scores = []
meteor_scores = []
exact_match = []
for pair in tqdm(pairs):
    _, reference, hypothesis = pair
    # Skip empty pairs
    if not reference.strip() or not hypothesis.strip():
        continue
    # BLEU
    bleu_score = bleu.compute(predictions=[hypothesis], references=[reference])
    bleu_scores.append(bleu_score["bleu"])
    # ROUGE
    rouge_score = rouge.compute(predictions=[hypothesis], references=[reference])
    rouge_scores.append(rouge_score["rougeL"])
    # METEOR
    meteor_score = meteor.compute(predictions=[hypothesis], references=[reference])
    meteor_scores.append(meteor_score["meteor"])
    # Exact match
    exact_match.append(hypothesis.strip() == reference.strip())

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
100%|██████████| 200/200 [00:32<00:00,  6.14it/s]


In [27]:
def safe_mean(lst):
    return np.mean(lst) if lst else float('nan')
print(f"BLEU Score (avg): {safe_mean(bleu_scores)}")
print(f"ROUGE-1 F1 (avg): {safe_mean(rouge_scores)}")
print(f"meteor_scores F1 (avg): {safe_mean(meteor_scores)}")
print(f"Exact Match (avg): {safe_mean(exact_match)}")

BLEU Score (avg): 0.013239490253819052
ROUGE-1 F1 (avg): 0.14152846905933766
meteor_scores F1 (avg): 0.20451203177807414
Exact Match (avg): 0.0


In [28]:
filename = "pairs_tuned_clean_full_gan.json"
with open(filename, "r", encoding="utf-8") as f:
    pairs = json.load(f)

In [29]:
bleu_scores = []
rouge_scores = []
meteor_scores = []
exact_match = [] # i will be very surprised if this ever goes above 0
for pair in tqdm(pairs):
    _, reference, hypothesis = pair
    if not reference.strip() or not hypothesis.strip():
        continue
    bleu_score = bleu.compute(predictions=[hypothesis], references=[reference])
    bleu_scores.append(bleu_score["bleu"])
    rouge_score = rouge.compute(predictions=[hypothesis], references=[reference])
    rouge_scores.append(rouge_score["rougeL"])
    meteor_score = meteor.compute(predictions=[hypothesis], references=[reference])
    meteor_scores.append(meteor_score["meteor"])
    exact_match.append(hypothesis.strip() == reference.strip())

100%|██████████| 200/200 [00:29<00:00,  6.85it/s]


In [30]:
def safe_mean(lst):
    return np.mean(lst) if lst else float('nan')
print(f"BLEU Score (avg): {safe_mean(bleu_scores)}")
print(f"ROUGE-1 F1 (avg): {safe_mean(rouge_scores)}")
print(f"meteor_scores F1 (avg): {safe_mean(meteor_scores)}")
print(f"Exact Match (avg): {safe_mean(exact_match)}")

BLEU Score (avg): 0.013239490253819052
ROUGE-1 F1 (avg): 0.14152846905933766
meteor_scores F1 (avg): 0.20451203177807414
Exact Match (avg): 0.0


In [ ]:
!du -sh /content/gan_fine_tuned_model/model-00001-of-00003.safetensors

In [31]:
!zip -r gan_fine_tuned_model.zip . -i ./gan_fine_tuned_model

	zip warning: zip file empty
